In [1]:
import pandas as pd
import glob as glob

## Importing Statcast Data

Here we do some initial data treatment.

In [2]:
# Import Statcast Data

for year in range(2008,2022):
    df = pd.read_csv(f'Statcast Raw Data/{year}.csv')
    df = df.rename({' first_name': 'first_name'}, axis = 1)
    df['first_name'] = df['first_name'].str.lstrip()
    df['full_name'] = df[['first_name','last_name']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
    df = df[['last_name',
         'first_name',
         'full_name',
         'player_id',
         'year',
         'player_age',
         'b_ab',
         'b_total_pa',
         'b_total_hits',
         'b_single',
         'b_double',
         'b_triple',
         'b_home_run',
         'b_strikeout',
         'b_walk',
         'b_k_percent',
         'b_bb_percent',
         'batting_avg',
         'slg_percent',
         'on_base_percent',
         'on_base_plus_slg']]
    df['full_name'] = df['full_name'].replace({' Jr.':'', '-':' ', ' III':''},regex=True)     
               
   # df.to_csv(f'Initial Treated Data/{year}_initial.csv')

Dropping some of the columns with Nan values. We won't drop the player_id column because we will use that as our index.
We will fill the blanks with the correct player ID.

### Players' ID source #1 - Statcast Database

The source we'll use to create a primary ID database is the Statcast data. From 2015 to 2021, all players have their respective ID listed.

In [3]:
path = 'Initial Treated Data'
all_files = glob.glob(path + "/*initial.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

# Extracting players IDs to create a primary database

p_id_db = frame[['full_name', 'player_id']]
p_id_db = p_id_db[p_id_db['player_id'].notna()]

In [4]:
frame = frame.drop(columns = 'Unnamed: 0')

Considering we might have two different players with the same name and without id, let's delete potential duplicates based on stats

In [5]:
stats_cols = ['full_name',
         'year',
         'player_age',
         'b_ab',
         'b_total_pa',
         'b_total_hits',
         'b_single',
         'b_double',
         'b_triple',
         'b_home_run',
         'b_strikeout',
         'b_walk',
         'b_k_percent',
         'b_bb_percent',
         'batting_avg',
         'slg_percent',
         'on_base_percent',
         'on_base_plus_slg']

frame = frame.drop_duplicates(subset=stats_cols, keep='first')

### Dataframe only with players without ID

In [6]:
with_id = frame[frame['player_id'].notna()]
with_id

last_name first_name       full_name  player_id  year  player_age  b_ab  \
5018      Colon    Bartolo   Bartolo Colon   112526.0  2015          42    58   
5019     Hunter      Torii    Torii Hunter   116338.0  2015          40   521   
5020      Ortiz      David     David Ortiz   120074.0  2015          40   528   
5021  Rodriguez       Alex  Alex Rodriguez   121347.0  2015          40   523   
5022    Ramirez     Aramis  Aramis Ramirez   133380.0  2015          37   475   
...         ...        ...             ...        ...   ...         ...   ...   
9806       Gray     Josiah     Josiah Gray   680686.0  2021          24    22   
9807      Duran     Jarren    Jarren Duran   680776.0  2021          25   107   
9808    Jeffers       Ryan    Ryan Jeffers   680777.0  2021          24   267   
9809     Miller       Owen     Owen Miller   680911.0  2021          25   191   
9810     Vaughn     Andrew   Andrew Vaughn   683734.0  2021          23   417   

      b_total_pa  b_total_hits  b_single  ...  b_triple  b_home_run  \
5018          64             8         7  ...         0           0   
5019         567           125        81  ...         0          22   
5020         614           144        70  ...         0          37   
5021         620           131        75  ...         1          33   
5022         516           117        68  ...         1          17   
...          ...           ...       ...  ...       ...         ...   
9806          24             1         1  ...         0           0   
9807         112            23        16  ...         2           2   
9808         293            53        28  ...         1          14   
9809         202            39        27  ...         0           4   
9810         469            98        61  ...         0          15   

      b_strikeout  b_walk  b_k_percent  b_bb_percent  batting_avg  \
5018           24       0         37.5           0.0        0.138   
5019          105      35         18.5           6.2        0.240   
5020           95      77         15.5          12.5        0.273   
5021          145      84         23.4          13.5        0.250   
5022           68      31         13.2           6.0        0.246   
...           ...     ...          ...           ...          ...   
9806           12       1         50.0           4.2        0.045   
9807           40       4         35.7           3.6        0.215   
9808          108      22         36.9           7.5        0.199   
9809           54       9         26.7           4.5        0.204   
9810          101      41         21.5           8.7        0.235   

      slg_percent  on_base_percent  on_base_plus_slg  
5018        0.155            0.141             0.296  
5019        0.409            0.293             0.702  
5020        0.553            0.360             0.913  
5021        0.486            0.356             0.842  
5022        0.423            0.297             0.720  
...           ...              ...               ...  
9806        0.045            0.083             0.129  
9807        0.336            0.241             0.578  
9808        0.401            0.270             0.670  
9809        0.309            0.243             0.551  
9810        0.396            0.309             0.705  

[4793 rows x 21 columns]

### Dataframe only with players without ID

In [7]:
no_id = frame[frame['player_id'].isna()]
no_id

last_name first_name      full_name  player_id  year  player_age  \
0           Young     Delwyn   Delwyn Young        NaN  2008          26   
1        Gonzalez       Andy  Andy Gonzalez        NaN  2008          27   
2        Cruz Jr.       Jose      Jose Cruz        NaN  2008          34   
3     Griffey Jr.        Ken    Ken Griffey        NaN  2008          39   
4       Jones Jr.    Chipper  Chipper Jones        NaN  2008          36   
...           ...        ...            ...        ...   ...         ...   
5013        Davis        Ike      Ike Davis        NaN  2014          27   
5014        Colon    Bartolo  Bartolo Colon        NaN  2014          41   
5015     Bautista       Jose  Jose Bautista        NaN  2014          34   
5016       Adduci        Jim     Jim Adduci        NaN  2014          29   
5017        Abreu      Bobby    Bobby Abreu        NaN  2014          40   

      b_ab  b_total_pa  b_total_hits  b_single  ...  b_triple  b_home_run  \
0      126         143            31        21  ...         0           1   
1       24          30             5         4  ...         0           1   
2       49          60             6         5  ...         0           0   
3      490         575           122        73  ...         1          18   
4      439         534           160       113  ...         1          22   
...    ...         ...           ...       ...  ...       ...         ...   
5013   360         427            84        54  ...         0          11   
5014    62          69             2         1  ...         0           0   
5015   553         673           158        96  ...         0          35   
5016   101         114            17        13  ...         0           1   
5017   133         155            33        23  ...         0           1   

      b_strikeout  b_walk  b_k_percent  b_bb_percent  batting_avg  \
0              34      14         23.8           9.8        0.246   
1               5       6         16.7          20.0        0.208   
2               9      11         15.0          18.3        0.122   
3              89      78         15.5          13.6        0.249   
4              61      90         11.4          16.9        0.364   
...           ...     ...          ...           ...          ...   
5013           78      63         18.3          14.8        0.233   
5014           33       0         47.8           0.0        0.032   
5015           96     104         14.3          15.5        0.286   
5016           27      10         23.7           8.8        0.168   
5017           21      20         13.5          12.9        0.248   

      slg_percent  on_base_percent  on_base_plus_slg  
0           0.341            0.315             0.656  
1           0.333            0.367             0.700  
2           0.143            0.283             0.426  
3           0.424            0.353             0.778  
4           0.574            0.470             1.044  
...           ...              ...               ...  
5013        0.378            0.344             0.722  
5014        0.048            0.029             0.077  
5015        0.524            0.403             0.927  
5016        0.228            0.237             0.465  
5017        0.338            0.342             0.680  

[5018 rows x 21 columns]

### Players' ID source #2 - WAR Database

Our WAR database contains data from players since the 1960's, including all players IDs.

In [8]:
from unidecode import unidecode

bat_data_war = pd.read_csv('war_daily_bat.txt', sep=",", encoding = 'utf8')
bat_data_war = bat_data_war[bat_data_war['year_ID']>=2008]

In [9]:
bat_data_war = bat_data_war.rename({'mlb_ID': 'player_id', 'name_common': 'full_name', 'year_ID': 'year', 'age':'player_age','PA':'b_total_pa','lg_ID':'league'}, axis = 1)
bat_data_war['full_name'] = bat_data_war['full_name'].apply(unidecode)

player_id_bat_war = bat_data_war[['full_name','player_id', 'year','b_total_pa']]
player_id_bat_war.reset_index(drop = True)
player_id_bat_war = player_id_bat_war.drop_duplicates()
player_id_bat_war.sort_values(by='full_name')
player_id_bat_war.to_csv('player_id_bat_war.csv')

original_names = bat_data_war[['player_id','full_name']].drop_duplicates()
original_names['full_name'] = original_names['full_name'].replace({'AJ Pollock': 'A.J. Pollock'}, regex = True)
#original_names.to_csv('original_names.csv')

After some tries to get the most of the WAR database, we noticed that some values in the 'full_name' column were affecting the code effectiveness. With this in mind, the changes below were added:

In [10]:
player_id_bat_war['full_name'] = player_id_bat_war['full_name'].replace({
    'AJ Pollock':'A.J. Pollock', 
    'Chin-lung Hu':'Chin-Lung Hu',
    'Henderson Alvarez III':'Henderson Alvarez',
    'Hung-Chih Kuo':'Hong-Chih Kuo',
    'Hyun Jin Ryu':'Hyun-Jin Ryu',
    'JB Shuck':'J.B. Shuck',
    'Luis Cruz':'Luis Alfonso Cruz',
    'Luis Rodriguez':'Luis Antonio Rodriguez',
    'Mike Ryan':'Michael Ryan',
    'Mike Difelice':'Mike DiFelice',
    'Osvaldo Martinez': 'Ozzie Martinez',
    'Rick van den Hurk': 'Rick Van Den Hurk',
    'Russ Mitchell':'Russell Mitchell',
    'Val Pascucci':'Valentino Pascucci',
    'Vidal Nuno III': 'Vidal Nuno',
    'Wil Ledezma':'Wilfredo Ledezma',
    ' Jr.':'',
    ' Jr':',',
    ' III':'',
    '-':' ' 
}, regex = True)
player_id_bat_war['full_name'] = player_id_bat_war['full_name'].str.rstrip()

In [11]:
player_id_bat_war

full_name  player_id  year  b_total_pa
3       David Aardsma   430911.0  2008         1.0
4       David Aardsma   430911.0  2009         0.0
5       David Aardsma   430911.0  2010         0.0
6       David Aardsma   430911.0  2012         0.0
7       David Aardsma   430911.0  2013         0.0
...               ...        ...   ...         ...
118754    Mike Zunino   572287.0  2020        84.0
118755    Mike Zunino   572287.0  2021       375.0
118787      Tony Zych   543964.0  2015         0.0
118788      Tony Zych   543964.0  2016         0.0
118789      Tony Zych   543964.0  2017         0.0

[19722 rows x 4 columns]

## Combining the datasets

#### First filling the Nan player ids with data from the Statcast dataset

Here we get three versions of the players' id database from the WAR data. As you can see, each version of the database has less details. This might help us to address data with different details.

In [12]:
player_id_bat_war

full_name  player_id  year  b_total_pa
3       David Aardsma   430911.0  2008         1.0
4       David Aardsma   430911.0  2009         0.0
5       David Aardsma   430911.0  2010         0.0
6       David Aardsma   430911.0  2012         0.0
7       David Aardsma   430911.0  2013         0.0
...               ...        ...   ...         ...
118754    Mike Zunino   572287.0  2020        84.0
118755    Mike Zunino   572287.0  2021       375.0
118787      Tony Zych   543964.0  2015         0.0
118788      Tony Zych   543964.0  2016         0.0
118789      Tony Zych   543964.0  2017         0.0

[19722 rows x 4 columns]

In [13]:
player_id_bat_war_2 = player_id_bat_war.drop(columns = ['b_total_pa'])
player_id_bat_war_2

full_name  player_id  year
3       David Aardsma   430911.0  2008
4       David Aardsma   430911.0  2009
5       David Aardsma   430911.0  2010
6       David Aardsma   430911.0  2012
7       David Aardsma   430911.0  2013
...               ...        ...   ...
118754    Mike Zunino   572287.0  2020
118755    Mike Zunino   572287.0  2021
118787      Tony Zych   543964.0  2015
118788      Tony Zych   543964.0  2016
118789      Tony Zych   543964.0  2017

[19722 rows x 3 columns]

First we merge the data using the most variables as binding points as we possibly can.

In [14]:
df2t = pd.merge(no_id, player_id_bat_war, how = 'left', on=['full_name','year','b_total_pa'])
df2t = df2t.drop(columns = 'player_id_x')
df2t = df2t.rename({'player_id_y':'player_id'},axis=1)
df2t = df2t[['last_name',
         'first_name',
         'full_name',
         'player_id',
         'year',
         'player_age',
         'b_ab',
         'b_total_pa',
         'b_total_hits',
         'b_single',
         'b_double',
         'b_triple',
         'b_home_run',
         'b_strikeout',
         'b_walk',
         'b_k_percent',
         'b_bb_percent',
         'batting_avg',
         'slg_percent',
         'on_base_percent',
         'on_base_plus_slg']]
df2t

last_name first_name      full_name  player_id  year  player_age  \
0           Young     Delwyn   Delwyn Young   434704.0  2008          26   
1        Gonzalez       Andy  Andy Gonzalez   462413.0  2008          27   
2        Cruz Jr.       Jose      Jose Cruz   112903.0  2008          34   
3     Griffey Jr.        Ken    Ken Griffey        NaN  2008          39   
4       Jones Jr.    Chipper  Chipper Jones   116706.0  2008          36   
...           ...        ...            ...        ...   ...         ...   
5013        Davis        Ike      Ike Davis        NaN  2014          27   
5014        Colon    Bartolo  Bartolo Colon   112526.0  2014          41   
5015     Bautista       Jose  Jose Bautista   430832.0  2014          34   
5016       Adduci        Jim     Jim Adduci   451192.0  2014          29   
5017        Abreu      Bobby    Bobby Abreu   110029.0  2014          40   

      b_ab  b_total_pa  b_total_hits  b_single  ...  b_triple  b_home_run  \
0      126         143            31        21  ...         0           1   
1       24          30             5         4  ...         0           1   
2       49          60             6         5  ...         0           0   
3      490         575           122        73  ...         1          18   
4      439         534           160       113  ...         1          22   
...    ...         ...           ...       ...  ...       ...         ...   
5013   360         427            84        54  ...         0          11   
5014    62          69             2         1  ...         0           0   
5015   553         673           158        96  ...         0          35   
5016   101         114            17        13  ...         0           1   
5017   133         155            33        23  ...         0           1   

      b_strikeout  b_walk  b_k_percent  b_bb_percent  batting_avg  \
0              34      14         23.8           9.8        0.246   
1               5       6         16.7          20.0        0.208   
2               9      11         15.0          18.3        0.122   
3              89      78         15.5          13.6        0.249   
4              61      90         11.4          16.9        0.364   
...           ...     ...          ...           ...          ...   
5013           78      63         18.3          14.8        0.233   
5014           33       0         47.8           0.0        0.032   
5015           96     104         14.3          15.5        0.286   
5016           27      10         23.7           8.8        0.168   
5017           21      20         13.5          12.9        0.248   

      slg_percent  on_base_percent  on_base_plus_slg  
0           0.341            0.315             0.656  
1           0.333            0.367             0.700  
2           0.143            0.283             0.426  
3           0.424            0.353             0.778  
4           0.574            0.470             1.044  
...           ...              ...               ...  
5013        0.378            0.344             0.722  
5014        0.048            0.029             0.077  
5015        0.524            0.403             0.927  
5016        0.228            0.237             0.465  
5017        0.338            0.342             0.680  

[5018 rows x 21 columns]

Let's separate the rows with and without player id after this iteration

In [15]:
df2t_with_id = df2t[df2t['player_id'].notna()]

In [16]:
df2t_without_id = df2t[df2t['player_id'].isna()]
df2t_without_id

last_name first_name          full_name  player_id  year  player_age  \
3     Griffey Jr.        Ken        Ken Griffey        NaN  2008          39   
43       Gonzalez    Alberto   Alberto Gonzalez        NaN  2008          25   
50      Bonifacio     Emilio   Emilio Bonifacio        NaN  2008          23   
66           Moss    Brandon       Brandon Moss        NaN  2008          25   
149        Owings      Micah       Micah Owings        NaN  2008          26   
...           ...        ...                ...        ...   ...         ...   
4991   Pierzynski       A.J.    A.J. Pierzynski        NaN  2014          38   
5004        Kelly        Joe          Joe Kelly        NaN  2014          26   
5006       Ibanez       Raul        Raul Ibanez        NaN  2014          42   
5008    Hernandez    Roberto  Roberto Hernandez        NaN  2014          34   
5013        Davis        Ike          Ike Davis        NaN  2014          27   

      b_ab  b_total_pa  b_total_hits  b_single  ...  b_triple  b_home_run  \
3      490         575           122        73  ...         1          18   
43     101         112            26        17  ...         0           1   
50     169         186            41        30  ...         5           0   
66     236         263            58        32  ...         3           8   
149     56          62            17        13  ...         0           1   
...    ...         ...           ...       ...  ...       ...         ...   
4991   338         362            85        67  ...         1           5   
5004    16          18             4         3  ...         0           0   
5006   246         280            41        25  ...         3           5   
5008    49          58             4         4  ...         0           0   
5013   360         427            84        54  ...         0          11   

      b_strikeout  b_walk  b_k_percent  b_bb_percent  batting_avg  \
3              89      78         15.5          13.6        0.249   
43             14       8         12.5           7.1        0.257   
50             46      14         24.7           7.5        0.243   
66             70      21         26.6           8.0        0.246   
149            21       5         33.9           8.1        0.304   
...           ...     ...          ...           ...          ...   
4991           54      14         14.9           3.9        0.251   
5004            2       0         11.1           0.0        0.250   
5006           59      33         21.1          11.8        0.167   
5008           24       0         41.4           0.0        0.082   
5013           78      63         18.3          14.8        0.233   

      slg_percent  on_base_percent  on_base_plus_slg  
3           0.424            0.353             0.778  
43          0.366            0.313             0.679  
50          0.337            0.296             0.633  
66          0.436            0.304             0.741  
149         0.411            0.355             0.766  
...           ...              ...               ...  
4991        0.337            0.287             0.625  
5004        0.313            0.278             0.590  
5006        0.285            0.264             0.549  
5008        0.082            0.069             0.151  
5013        0.378            0.344             0.722  

[454 rows x 21 columns]

Now let's try getting some more player ids with the less restrictive dataset

In [17]:
df3t = pd.merge(df2t_without_id, player_id_bat_war_2, how = 'left', on=['full_name','year'])
df3t = df3t.drop_duplicates()
df3t = df3t.drop(columns=['player_id_x'],axis=1)
df3t = df3t.rename({'player_id_y':'player_id'},axis=1)
df3t = df3t[['last_name',
         'first_name',
         'full_name',
         'player_id',
         'year',
         'player_age',
         'b_ab',
         'b_total_pa',
         'b_total_hits',
         'b_single',
         'b_double',
         'b_triple',
         'b_home_run',
         'b_strikeout',
         'b_walk',
         'b_k_percent',
         'b_bb_percent',
         'batting_avg',
         'slg_percent',
         'on_base_percent',
         'on_base_plus_slg']]
df3t

last_name first_name          full_name  player_id  year  player_age  \
0    Griffey Jr.        Ken        Ken Griffey   115135.0  2008          39   
2       Gonzalez    Alberto   Alberto Gonzalez   471868.0  2008          25   
4      Bonifacio     Emilio   Emilio Bonifacio   466988.0  2008          23   
6           Moss    Brandon       Brandon Moss   461235.0  2008          25   
8         Owings      Micah       Micah Owings   452249.0  2008          26   
..           ...        ...                ...        ...   ...         ...   
879   Pierzynski       A.J.    A.J. Pierzynski   150229.0  2014          38   
881        Kelly        Joe          Joe Kelly   523260.0  2014          26   
883       Ibanez       Raul        Raul Ibanez   116380.0  2014          42   
885    Hernandez    Roberto  Roberto Hernandez   433584.0  2014          34   
887        Davis        Ike          Ike Davis   477195.0  2014          27   

     b_ab  b_total_pa  b_total_hits  b_single  ...  b_triple  b_home_run  \
0     490         575           122        73  ...         1          18   
2     101         112            26        17  ...         0           1   
4     169         186            41        30  ...         5           0   
6     236         263            58        32  ...         3           8   
8      56          62            17        13  ...         0           1   
..    ...         ...           ...       ...  ...       ...         ...   
879   338         362            85        67  ...         1           5   
881    16          18             4         3  ...         0           0   
883   246         280            41        25  ...         3           5   
885    49          58             4         4  ...         0           0   
887   360         427            84        54  ...         0          11   

     b_strikeout  b_walk  b_k_percent  b_bb_percent  batting_avg  slg_percent  \
0             89      78         15.5          13.6        0.249        0.424   
2             14       8         12.5           7.1        0.257        0.366   
4             46      14         24.7           7.5        0.243        0.337   
6             70      21         26.6           8.0        0.246        0.436   
8             21       5         33.9           8.1        0.304        0.411   
..           ...     ...          ...           ...          ...          ...   
879           54      14         14.9           3.9        0.251        0.337   
881            2       0         11.1           0.0        0.250        0.313   
883           59      33         21.1          11.8        0.167        0.285   
885           24       0         41.4           0.0        0.082        0.082   
887           78      63         18.3          14.8        0.233        0.378   

     on_base_percent  on_base_plus_slg  
0              0.353             0.778  
2              0.313             0.679  
4              0.296             0.633  
6              0.304             0.741  
8              0.355             0.766  
..               ...               ...  
879            0.287             0.625  
881            0.278             0.590  
883            0.264             0.549  
885            0.069             0.151  
887            0.344             0.722  

[455 rows x 21 columns]

In [18]:
df3t_with_id = df3t[df3t['player_id'].notna()]

In [19]:
df3t_without_id = df3t[df3t['player_id'].isna()]
df3t_without_id

last_name       first_name                  full_name  player_id  year  \
20        Morse          Michael              Michael Morse        NaN  2008   
132       Morse          Michael              Michael Morse        NaN  2009   
241       Niese         Jonathon             Jonathon Niese        NaN  2010   
246    Tolleson            Steve             Steve Tolleson        NaN  2010   
288       Morse          Michael              Michael Morse        NaN  2010   
375       Shuck               JB                   JB Shuck        NaN  2011   
384       Niese         Jonathon             Jonathon Niese        NaN  2011   
385       Field            Tommy                Tommy Field        NaN  2011   
410       Morse          Michael              Michael Morse        NaN  2011   
481   Rodriguez  Henry Alejandro  Henry Alejandro Rodriguez        NaN  2012   
496        Aoki        Norichika             Norichika Aoki        NaN  2012   
501       Niese         Jonathon             Jonathon Niese        NaN  2012   
502    Tolleson            Steve             Steve Tolleson        NaN  2012   
550       Morse          Michael              Michael Morse        NaN  2012   
615  den Dekker          Matthew         Matthew den Dekker        NaN  2013   
616         Ryu         Hyun Jin               Hyun Jin Ryu        NaN  2013   
619     Pollock               AJ                 AJ Pollock        NaN  2013   
622       Shuck               JB                   JB Shuck        NaN  2013   
629   Rodriguez  Henry Alejandro  Henry Alejandro Rodriguez        NaN  2013   
638        Aoki        Norichika             Norichika Aoki        NaN  2013   
648       Niese         Jonathon             Jonathon Niese        NaN  2013   
651       Field            Tommy                Tommy Field        NaN  2013   
656       Perez      Juan Carlos          Juan Carlos Perez        NaN  2013   
697       Morse          Michael              Michael Morse        NaN  2013   
748       Lopez            Raffy                Raffy Lopez        NaN  2014   
753         Ryu         Hyun-Jin               Hyun Jin Ryu        NaN  2014   
760  den Dekker          Matthew         Matthew den Dekker        NaN  2014   
787        Aoki        Norichika             Norichika Aoki        NaN  2014   
788       Niese         Jonathon             Jonathon Niese        NaN  2014   
789    Tolleson            Steve             Steve Tolleson        NaN  2014   
794       Perez      Juan Carlos          Juan Carlos Perez        NaN  2014   
855       Morse          Michael              Michael Morse        NaN  2014   

     player_age  b_ab  b_total_pa  b_total_hits  b_single  ...  b_triple  \
20           26     9          11             2         1  ...         0   
132          27    52          55            13         7  ...         0   
241          24    53          66            10         8  ...         0   
246          27    49          53            14        10  ...         0   
288          28   266         293            77        48  ...         2   
375          24    81          92            22        19  ...         1   
384          25    49          60             3         2  ...         1   
385          24    48          51            13        13  ...         0   
410          29   522         575           158        91  ...         0   
481          22    14          16             3         2  ...         0   
496          30   520         588           150        99  ...         4   
501          26    55          68            12        12  ...         0   
502          29    71          76            13         8  ...         0   
550          30   406         430           118        82  ...         1   
615          26    58          63            12        10  ...         0   
616          26    58          66            12         8  ...         1   
619          26   443         482           119        78  ...         5 

Now, for the remaining 32 datapoints without ID, let's apply the player ID data from the Statcast dataframe. At this point, the problem to correlate the dataset and the players' ID from the WAR table is how the names are written. 

In [20]:
df4t = pd.merge(df3t_without_id, p_id_db, how = 'left', on=['full_name'])
df4t = df4t.drop_duplicates()
df4t = df4t.drop(columns=['player_id_x'],axis=1)
df4t = df4t.rename({'player_id_y':'player_id'},axis=1)
df4t = df4t[['last_name',
         'first_name',
         'full_name',
         'player_id',
         'year',
         'player_age',
         'b_ab',
         'b_total_pa',
         'b_total_hits',
         'b_single',
         'b_double',
         'b_triple',
         'b_home_run',
         'b_strikeout',
         'b_walk',
         'b_k_percent',
         'b_bb_percent',
         'batting_avg',
         'slg_percent',
         'on_base_percent',
         'on_base_plus_slg']]
df4t

last_name       first_name                  full_name  player_id  year  \
0        Morse          Michael              Michael Morse   434604.0  2008   
2        Morse          Michael              Michael Morse   434604.0  2009   
4        Niese         Jonathon             Jonathon Niese   477003.0  2010   
6     Tolleson            Steve             Steve Tolleson   476270.0  2010   
7        Morse          Michael              Michael Morse   434604.0  2010   
9        Shuck               JB                   JB Shuck   543776.0  2011   
13       Niese         Jonathon             Jonathon Niese   477003.0  2011   
15       Field            Tommy                Tommy Field   474494.0  2011   
16       Morse          Michael              Michael Morse   434604.0  2011   
18   Rodriguez  Henry Alejandro  Henry Alejandro Rodriguez        NaN  2012   
19        Aoki        Norichika             Norichika Aoki   493114.0  2012   
22       Niese         Jonathon             Jonathon Niese   477003.0  2012   
24    Tolleson            Steve             Steve Tolleson   476270.0  2012   
25       Morse          Michael              Michael Morse   434604.0  2012   
27  den Dekker          Matthew         Matthew den Dekker   544925.0  2013   
30         Ryu         Hyun Jin               Hyun Jin Ryu   547943.0  2013   
33     Pollock               AJ                 AJ Pollock   572041.0  2013   
39       Shuck               JB                   JB Shuck   543776.0  2013   
43   Rodriguez  Henry Alejandro  Henry Alejandro Rodriguez        NaN  2013   
44        Aoki        Norichika             Norichika Aoki   493114.0  2013   
47       Niese         Jonathon             Jonathon Niese   477003.0  2013   
49       Field            Tommy                Tommy Field   474494.0  2013   
50       Perez      Juan Carlos          Juan Carlos Perez   543633.0  2013   
51       Morse          Michael              Michael Morse   434604.0  2013   
53       Lopez            Raffy                Raffy Lopez   607257.0  2014   
55         Ryu         Hyun-Jin               Hyun Jin Ryu   547943.0  2014   
58  den Dekker          Matthew         Matthew den Dekker   544925.0  2014   
61        Aoki        Norichika             Norichika Aoki   493114.0  2014   
64       Niese         Jonathon             Jonathon Niese   477003.0  2014   
66    Tolleson            Steve             Steve Tolleson   476270.0  2014   
67       Perez      Juan Carlos          Juan Carlos Perez   543633.0  2014   
68       Morse          Michael              Michael Morse   434604.0  2014   

    player_age  b_ab  b_total_pa  b_total_hits  b_single  ...  b_triple  \
0           26     9          11             2         1  ...         0   
2           27    52          55            13         7  ...         0   
4           24    53          66            10         8  ...         0   
6           27    49          53            14        10  ...         0   
7           28   266         293            77        48  ...         2   
9           24    81          92            22        19  ...         1   
13          25    49          60             3         2  ...         1   
15          24    48          51            13        13  ...         0   
16          29   522         575           158        91  ...         0   
18          22    14          16             3         2  ...         0   
19          30   520         588           150        99  ...         4   
22          26    55          68            12        12  ...         0   
24          29    71          76            13         8  ...         0   
25          30   406         430           118        82  ...         1   
27          26    58          63            12        10  ...         0   
30          26    58          66            12         8  ...         1   
33          26   443         482           119        78  ...         5   
39          26   437         478           128 

In [21]:
df4t_with_id = df4t[df4t['player_id'].notna()]

In [22]:
df4t_without_id = df4t[df4t['player_id'].isna()]
df4t_without_id

last_name       first_name                  full_name  player_id  year  \
18  Rodriguez  Henry Alejandro  Henry Alejandro Rodriguez        NaN  2012   
43  Rodriguez  Henry Alejandro  Henry Alejandro Rodriguez        NaN  2013   

    player_age  b_ab  b_total_pa  b_total_hits  b_single  ...  b_triple  \
18          22    14          16             3         2  ...         0   
43          23     9          10             1         1  ...         0   

    b_home_run  b_strikeout  b_walk  b_k_percent  b_bb_percent  batting_avg  \
18           0            2       2         12.5          12.5        0.214   
43           0            4       1         40.0          10.0        0.111   

    slg_percent  on_base_percent  on_base_plus_slg  
18        0.286            0.313             0.598  
43        0.111            0.200             0.311  

[2 rows x 21 columns]

Now we can see we only got one player left!

Manually checking the databases, we observe that there are two players named Henry Rodriguez that played in the MLB between 2008 and 2021. Searching online the player's ID from the WAR database, we discovered that the ID 514719 corresponds to our missing player! (https://www.thebaseballcube.com/content/player/152792/). Therefore, let's add this player's ID manually!

In [23]:
player_id_bat_war.loc[player_id_bat_war['player_id'] == 514719, 'full_name'] = 'Henry Alejandro Rodriguez'

In [24]:
df5t = pd.merge(df4t_without_id, player_id_bat_war, how = 'left', on=['full_name','year','b_total_pa'])
df5t = df5t.drop_duplicates()
df5t = df5t.drop(columns=['player_id_x'],axis=1)
df5t = df5t.rename({'player_id_y':'player_id'},axis=1)
df5t = df5t[['last_name',
         'first_name',
         'full_name',
         'player_id',
         'year',
         'player_age',
         'b_ab',
         'b_total_pa',
         'b_total_hits',
         'b_single',
         'b_double',
         'b_triple',
         'b_home_run',
         'b_strikeout',
         'b_walk',
         'b_k_percent',
         'b_bb_percent',
         'batting_avg',
         'slg_percent',
         'on_base_percent',
         'on_base_plus_slg']]
df5t

last_name       first_name                  full_name  player_id  year  \
0  Rodriguez  Henry Alejandro  Henry Alejandro Rodriguez   514719.0  2012   
1  Rodriguez  Henry Alejandro  Henry Alejandro Rodriguez   514719.0  2013   

   player_age  b_ab  b_total_pa  b_total_hits  b_single  ...  b_triple  \
0          22    14          16             3         2  ...         0   
1          23     9          10             1         1  ...         0   

   b_home_run  b_strikeout  b_walk  b_k_percent  b_bb_percent  batting_avg  \
0           0            2       2         12.5          12.5        0.214   
1           0            4       1         40.0          10.0        0.111   

   slg_percent  on_base_percent  on_base_plus_slg  
0        0.286            0.313             0.598  
1        0.111            0.200             0.311  

[2 rows x 21 columns]

In [25]:
df5t_with_id = df5t[df5t['player_id'].notna()]

In [26]:
df5t_without_id = df5t[df5t['player_id'].isna()]
df5t_without_id

Empty DataFrame
Columns: [last_name, first_name, full_name, player_id, year, player_age, b_ab, b_total_pa, b_total_hits, b_single, b_double, b_triple, b_home_run, b_strikeout, b_walk, b_k_percent, b_bb_percent, batting_avg, slg_percent, on_base_percent, on_base_plus_slg]
Index: []

[0 rows x 21 columns]

Great! Now all our players have their respective IDs!

#### Concatenate all the total dataframe pieces

In [27]:
pieces = [with_id, df2t_with_id, df3t_with_id, df3t_with_id, df5t_with_id]

complete_data = pd.concat(pieces).reset_index(drop=True)
complete_data

last_name       first_name                  full_name  player_id  year  \
0          Colon          Bartolo              Bartolo Colon   112526.0  2015   
1         Hunter            Torii               Torii Hunter   116338.0  2015   
2          Ortiz            David                David Ortiz   120074.0  2015   
3      Rodriguez             Alex             Alex Rodriguez   121347.0  2015   
4        Ramirez           Aramis             Aramis Ramirez   133380.0  2015   
...          ...              ...                        ...        ...   ...   
10200     Ibanez             Raul                Raul Ibanez   116380.0  2014   
10201  Hernandez          Roberto          Roberto Hernandez   433584.0  2014   
10202      Davis              Ike                  Ike Davis   477195.0  2014   
10203  Rodriguez  Henry Alejandro  Henry Alejandro Rodriguez   514719.0  2012   
10204  Rodriguez  Henry Alejandro  Henry Alejandro Rodriguez   514719.0  2013   

       player_age  b_ab  b_total_pa  b_total_hits  b_single  ...  b_triple  \
0              42    58          64             8         7  ...         0   
1              40   521         567           125        81  ...         0   
2              40   528         614           144        70  ...         0   
3              40   523         620           131        75  ...         1   
4              37   475         516           117        68  ...         1   
...           ...   ...         ...           ...       ...  ...       ...   
10200          42   246         280            41        25  ...         3   
10201          34    49          58             4         4  ...         0   
10202          27   360         427            84        54  ...         0   
10203          22    14          16             3         2  ...         0   
10204          23     9          10             1         1  ...         0   

       b_home_run  b_strikeout  b_walk  b_k_percent  b_bb_percent  \
0               0           24       0         37.5           0.0   
1              22          105      35         18.5           6.2   
2              37           95      77         15.5          12.5   
3              33          145      84         23.4          13.5   
4              17           68      31         13.2           6.0   
...           ...          ...     ...          ...           ...   
10200           5           59      33         21.1          11.8   
10201           0           24       0         41.4           0.0   
10202          11           78      63         18.3          14.8   
10203           0            2       2         12.5          12.5   
10204           0            4       1         40.0          10.0   

       batting_avg  slg_percent  on_base_percent  on_base_plus_slg  
0            0.138        0.155            0.141             0.296  
1            0.240        0.409            0.293             0.702  
2            0.273        0.553            0.360             0.913  
3            0.250        0.486            0.356             0.842  
4            0.246        0.423            0.297             0.720  
...            ...          ...              ...               ...  
10200        0.167        0.285            0.264             0.549  
10201        0.082        0.082            0.069             0.151  
10202        0.233        0.378            0.344             0.722  
10203        0.214        0.286            0.313             0.598  
10204        0.111        0.111            0.200             0.311  

[10205 rows x 21 columns]

#### Retrieve Original Names

To avoid naming errors (for example same players named differently) we will convert the player's name in the 'with_id' table to the name on the original names file

In [28]:
complete_data_2 = pd.merge(complete_data, original_names, how = 'left', on=['player_id'])
complete_data_2.drop(columns = 'full_name_x',axis =1)
complete_data_2 = complete_data_2.rename({'full_name_y':'full_name'},axis=1)
complete_data_2 = complete_data_2[['last_name',
         'first_name',
         'full_name',
         'player_id',
         'year',
         'player_age',
         'b_ab',
         'b_total_pa',
         'b_total_hits',
         'b_single',
         'b_double',
         'b_triple',
         'b_home_run',
         'b_strikeout',
         'b_walk',
         'b_k_percent',
         'b_bb_percent',
         'batting_avg',
         'slg_percent',
         'on_base_percent',
         'on_base_plus_slg']]


In [29]:
stats_cols = ['full_name',
         'year',
         'player_age',
         'b_ab',
         'b_total_pa',
         'b_total_hits',
         'b_single',
         'b_double',
         'b_triple',
         'b_home_run',
         'b_strikeout',
         'b_walk',
         'b_k_percent',
         'b_bb_percent',
         'batting_avg',
         'slg_percent',
         'on_base_percent',
         'on_base_plus_slg']

complete_data_2 = complete_data_2.drop_duplicates(subset=stats_cols, keep='first')
complete_data_2

last_name       first_name          full_name  player_id  year  \
0          Colon          Bartolo      Bartolo Colon   112526.0  2015   
1         Hunter            Torii       Torii Hunter   116338.0  2015   
2          Ortiz            David        David Ortiz   120074.0  2015   
3      Rodriguez             Alex     Alex Rodriguez   121347.0  2015   
4        Ramirez           Aramis     Aramis Ramirez   133380.0  2015   
...          ...              ...                ...        ...   ...   
9777      Ibanez             Raul        Raul Ibanez   116380.0  2014   
9778   Hernandez          Roberto  Roberto Hernandez   433584.0  2014   
9779       Davis              Ike          Ike Davis   477195.0  2014   
10203  Rodriguez  Henry Alejandro    Henry Rodriguez   514719.0  2012   
10204  Rodriguez  Henry Alejandro    Henry Rodriguez   514719.0  2013   

       player_age  b_ab  b_total_pa  b_total_hits  b_single  ...  b_triple  \
0              42    58          64             8         7  ...         0   
1              40   521         567           125        81  ...         0   
2              40   528         614           144        70  ...         0   
3              40   523         620           131        75  ...         1   
4              37   475         516           117        68  ...         1   
...           ...   ...         ...           ...       ...  ...       ...   
9777           42   246         280            41        25  ...         3   
9778           34    49          58             4         4  ...         0   
9779           27   360         427            84        54  ...         0   
10203          22    14          16             3         2  ...         0   
10204          23     9          10             1         1  ...         0   

       b_home_run  b_strikeout  b_walk  b_k_percent  b_bb_percent  \
0               0           24       0         37.5           0.0   
1              22          105      35         18.5           6.2   
2              37           95      77         15.5          12.5   
3              33          145      84         23.4          13.5   
4              17           68      31         13.2           6.0   
...           ...          ...     ...          ...           ...   
9777            5           59      33         21.1          11.8   
9778            0           24       0         41.4           0.0   
9779           11           78      63         18.3          14.8   
10203           0            2       2         12.5          12.5   
10204           0            4       1         40.0          10.0   

       batting_avg  slg_percent  on_base_percent  on_base_plus_slg  
0            0.138        0.155            0.141             0.296  
1            0.240        0.409            0.293             0.702  
2            0.273        0.553            0.360             0.913  
3            0.250        0.486            0.356             0.842  
4            0.246        0.423            0.297             0.720  
...            ...          ...              ...               ...  
9777         0.167        0.285            0.264             0.549  
9778         0.082        0.082            0.069             0.151  
9779         0.233        0.378            0.344             0.722  
10203        0.214        0.286            0.313             0.598  
10204        0.111        0.111            0.200             0.311  

[9781 rows x 21 columns]

In [30]:
complete_data_2 = complete_data_2[['full_name',
         'player_id',
         'year',
         'player_age',
         'b_ab',
         'b_total_pa',
         'b_total_hits',
         'b_single',
         'b_double',
         'b_triple',
         'b_home_run',
         'b_strikeout',
         'b_walk',
         'b_k_percent',
         'b_bb_percent',
         'batting_avg',
         'slg_percent',
         'on_base_percent',
         'on_base_plus_slg']]

#### Now let's check for if we have different players with the same name (by same name we mean the same name in our dataframe)

In [31]:
fr_gb = frame.groupby('full_name')['player_id'].nunique().reset_index()
fr_gb = fr_gb.sort_values(by = 'player_id', ascending = False)
fr_gb

full_name  player_id
599    Daniel Robertson          2
0          A.J. Burnett          1
1107    Jeff Samardzija          1
1121  Jeremy Hellickson          1
1994         Ryan Court          1
...                 ...        ...
1165         Joe Mather          0
1164        Joe Mahoney          0
1163      Joe Koshansky          0
1162          Joe Kelly          0
2412      Zoilo Almonte          0

[2413 rows x 2 columns]

We must check what is happening to Daniel Robertson. But first, let's do the same check on the newer database:

In [32]:
cd2_gb = complete_data_2.groupby('full_name')['player_id'].nunique().reset_index()
cd2_gb = cd2_gb.sort_values(by = 'player_id', ascending = False)
cd2_gb

full_name  player_id
1494        Luis Jimenez          2
599     Daniel Robertson          2
29            Adam Eaton          2
1486       Luis Castillo          2
730       Edgar Gonzalez          2
...                  ...        ...
809      Francisco Arcia          1
810   Francisco Cervelli          1
811     Francisco Lindor          1
812    Francisco Liriano          1
2412       Zoilo Almonte          1

[2413 rows x 2 columns]

Here we must check for the players names that have 2 ids.

In [33]:
names = cd2_gb.loc[cd2_gb['player_id']!=1]
names.sort_values(by = 'full_name')

full_name  player_id
29          Adam Eaton          2
311     Brian Anderson          2
449       Chris Carter          2
489        Chris Young          2
599   Daniel Robertson          2
730     Edgar Gonzalez          2
1300         Josh Bell          2
1486     Luis Castillo          2
1491     Luis Gonzalez          2
1494      Luis Jimenez          2

In [34]:
names_list = names['full_name'].to_list()
names_list

['Luis Jimenez',
 'Daniel Robertson',
 'Adam Eaton',
 'Luis Castillo',
 'Edgar Gonzalez',
 'Josh Bell',
 'Luis Gonzalez',
 'Chris Young',
 'Brian Anderson',
 'Chris Carter']

In [35]:
print(fr_gb.shape)
print(cd2_gb.shape)

(2413, 2)
(2413, 2)


We can observe that our datasets have the same amount of unique names.

We can check in the total batters data if these names have indeed 2 ids each (meaning different players with the same name):

In [36]:
bat_data_war['full_name'] = bat_data_war['full_name'].apply(unidecode)
bat_data_war_gb = bat_data_war.groupby('full_name')['player_id'].nunique().reset_index()
bat_data_war_gb.loc[bat_data_war_gb['full_name'].isin(names_list)]

full_name  player_id
57          Adam Eaton          2
556     Brian Anderson          2
822       Chris Carter          2
889        Chris Young          2
1089  Daniel Robertson          2
1347    Edgar Gonzalez          2
2385         Josh Bell          2
2755     Luis Castillo          2
2763     Luis Gonzalez          2
2766      Luis Jimenez          2

As we can see, all the names on our list have 2 ids, and therefore correspond to two players.

## Adding WAR and League Data

As there are players that are batters and pitchers, we need to get both WAR scores. We'll first start with batters WAR.

In [37]:
bat_data_war.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20223 entries, 3 to 118789
Data columns (total 49 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   full_name           20223 non-null  object 
 1   player_age          20223 non-null  float64
 2   player_id           20223 non-null  float64
 3   player_ID           20223 non-null  object 
 4   year                20223 non-null  int64  
 5   team_ID             20223 non-null  object 
 6   stint_ID            20223 non-null  int64  
 7   league              20223 non-null  object 
 8   b_total_pa          19521 non-null  float64
 9   G                   20223 non-null  int64  
 10  Inn                 20223 non-null  float64
 11  runs_bat            20223 non-null  float64
 12  runs_br             20223 non-null  float64
 13  runs_dp             20223 non-null  float64
 14  runs_field          20223 non-null  float64
 15  runs_infield        20223 non-null  float64
 16  run

We are interested in the 'WAR' and 'league' columns.

In [38]:
war_values_bat = bat_data_war[['full_name','player_id','year','league','WAR']]
war_values_bat

full_name  player_id  year league   WAR
3       David Aardsma   430911.0  2008     AL -0.02
4       David Aardsma   430911.0  2009     AL  0.00
5       David Aardsma   430911.0  2010     AL  0.00
6       David Aardsma   430911.0  2012     AL   NaN
7       David Aardsma   430911.0  2013     NL  0.00
...               ...        ...   ...    ...   ...
118754    Mike Zunino   572287.0  2020     AL -0.05
118755    Mike Zunino   572287.0  2021     AL  3.75
118787      Tony Zych   543964.0  2015     AL   NaN
118788      Tony Zych   543964.0  2016     AL   NaN
118789      Tony Zych   543964.0  2017     AL  0.00

[20223 rows x 5 columns]

Let's rename the WAR column to avoid future conflict.

In [39]:
war_values_bat = war_values_bat.rename({'WAR':'WAR_bat'},axis = 1)
war_values_bat

full_name  player_id  year league  WAR_bat
3       David Aardsma   430911.0  2008     AL    -0.02
4       David Aardsma   430911.0  2009     AL     0.00
5       David Aardsma   430911.0  2010     AL     0.00
6       David Aardsma   430911.0  2012     AL      NaN
7       David Aardsma   430911.0  2013     NL     0.00
...               ...        ...   ...    ...      ...
118754    Mike Zunino   572287.0  2020     AL    -0.05
118755    Mike Zunino   572287.0  2021     AL     3.75
118787      Tony Zych   543964.0  2015     AL      NaN
118788      Tony Zych   543964.0  2016     AL      NaN
118789      Tony Zych   543964.0  2017     AL     0.00

[20223 rows x 5 columns]

Now, let's get the pitchers WAR

In [40]:
pit_data_war = pd.read_csv('war_daily_pitch.txt', sep=",", encoding = 'utf8')
pit_data_war = pit_data_war[pit_data_war['year_ID']>=2008]

In [41]:
pit_data_war = pit_data_war.rename({'mlb_ID': 'player_id', 'name_common': 'full_name', 'year_ID': 'year', 'age':'player_age','PA':'b_total_pa','lg_ID':'league'}, axis = 1)

In [42]:
war_values_pit = pit_data_war[['full_name','player_id','year','league','WAR']]

In [43]:
war_values_pit = war_values_pit.rename({'WAR':'WAR_pit'},axis = 1)
war_values_pit

full_name  player_id  year league  WAR_pit
41725        David Aardsma   430911.0  2008     AL    -0.31
41726       Jeremy Accardo   435618.0  2008     AL    -0.52
41727       Alfredo Aceves   469686.0  2008     AL     1.34
41728        Nick Adenhart   444440.0  2008     AL    -0.38
41729  Jonathan Albaladejo   452293.0  2008     AL     0.21
...                    ...        ...   ...    ...      ...
52854      Jordan Yamamoto   657141.0  2021     NL    -0.22
52855         Eric Yardley   642233.0  2021     NL    -0.43
52856         Huascar Ynoa   660623.0  2021     NL     1.45
52857           Alex Young   622065.0  2021     NL    -0.86
52858    Jordan Zimmermann   519455.0  2021     NL    -0.08

[11134 rows x 5 columns]

Now, let's merge the two dataframes

In [44]:
war_values_merged = pd.merge(war_values_bat, war_values_pit, how = 'left', on=['full_name','player_id','year','league'])

In [45]:
war_values_merged

full_name  player_id  year league  WAR_bat  WAR_pit
0      David Aardsma   430911.0  2008     AL    -0.02    -0.31
1      David Aardsma   430911.0  2009     AL     0.00     1.69
2      David Aardsma   430911.0  2010     AL     0.00     0.68
3      David Aardsma   430911.0  2012     AL      NaN    -0.02
4      David Aardsma   430911.0  2013     NL     0.00    -0.13
...              ...        ...   ...    ...      ...      ...
21183    Mike Zunino   572287.0  2020     AL    -0.05      NaN
21184    Mike Zunino   572287.0  2021     AL     3.75      NaN
21185      Tony Zych   543964.0  2015     AL      NaN     0.42
21186      Tony Zych   543964.0  2016     AL      NaN     0.12
21187      Tony Zych   543964.0  2017     AL     0.00     1.07

[21188 rows x 6 columns]

Some players play exclusively in one position, so they will not have WAR results for the position they don't play.

In [46]:
war_values_merged.isnull().sum()

full_name       0
player_id       0
year            0
league          0
WAR_bat      2655
WAR_pit      9360
dtype: int64

In [47]:
war_values_merged = war_values_merged.fillna(0)

Now we can calculate a Total WAR value by adding the two WAR values

In [48]:
war_values_merged['Total_WAR'] = war_values_merged['WAR_bat'] + war_values_merged['WAR_pit']
war_values_merged

full_name  player_id  year league  WAR_bat  WAR_pit  Total_WAR
0      David Aardsma   430911.0  2008     AL    -0.02    -0.31      -0.33
1      David Aardsma   430911.0  2009     AL     0.00     1.69       1.69
2      David Aardsma   430911.0  2010     AL     0.00     0.68       0.68
3      David Aardsma   430911.0  2012     AL     0.00    -0.02      -0.02
4      David Aardsma   430911.0  2013     NL     0.00    -0.13      -0.13
...              ...        ...   ...    ...      ...      ...        ...
21183    Mike Zunino   572287.0  2020     AL    -0.05     0.00      -0.05
21184    Mike Zunino   572287.0  2021     AL     3.75     0.00       3.75
21185      Tony Zych   543964.0  2015     AL     0.00     0.42       0.42
21186      Tony Zych   543964.0  2016     AL     0.00     0.12       0.12
21187      Tony Zych   543964.0  2017     AL     0.00     1.07       1.07

[21188 rows x 7 columns]

In [49]:
war_values_merged.isnull().sum()

full_name    0
player_id    0
year         0
league       0
WAR_bat      0
WAR_pit      0
Total_WAR    0
dtype: int64

### Add the WAR and league results to our dataset

In [50]:
complete_data_and_war = pd.merge(complete_data_2, war_values_merged, how = 'left', on=['full_name','player_id','year'])
complete_data_and_war = complete_data_and_war[['full_name', 'player_id', 'year', 'player_age','league',
       'Total_WAR','WAR_bat', 'WAR_pit', 'b_ab', 'b_total_pa',
       'b_total_hits', 'b_single', 'b_double', 'b_triple', 'b_home_run',
       'b_strikeout', 'b_walk', 'b_k_percent', 'b_bb_percent', 'batting_avg',
       'slg_percent', 'on_base_percent', 'on_base_plus_slg']]
complete_data_and_war['full_name'] = complete_data_and_war['full_name'].apply(unidecode)
complete_data_and_war

full_name  player_id  year  player_age league  Total_WAR  \
0          Bartolo Colon   112526.0  2015          42     NL       1.22   
1           Torii Hunter   116338.0  2015          40     AL      -0.65   
2            David Ortiz   120074.0  2015          40     AL       3.10   
3         Alex Rodriguez   121347.0  2015          40     AL       2.95   
4         Aramis Ramirez   133380.0  2015          37     NL       0.00   
...                  ...        ...   ...         ...    ...        ...   
10904  Roberto Hernandez   433584.0  2014          34     NL      -0.79   
10905          Ike Davis   477195.0  2014          27     NL       0.34   
10906          Ike Davis   477195.0  2014          27     NL      -0.07   
10907    Henry Rodriguez   514719.0  2012          22     NL      -0.04   
10908    Henry Rodriguez   514719.0  2013          23     NL      -0.11   

       WAR_bat  WAR_pit  b_ab  b_total_pa  ...  b_triple  b_home_run  \
0        -0.07     1.29    58          64  ...         0           0   
1        -0.65     0.00   521         567  ...         0          22   
2         3.10     0.00   528         614  ...         0          37   
3         2.95     0.00   523         620  ...         1          33   
4         0.00     0.00   475         516  ...         1          17   
...        ...      ...   ...         ...  ...       ...         ...   
10904    -0.38    -0.41    49          58  ...         0           0   
10905     0.34     0.00   360         427  ...         0          11   
10906    -0.07     0.00   360         427  ...         0          11   
10907    -0.04     0.00    14          16  ...         0           0   
10908    -0.11     0.00     9          10  ...         0           0   

       b_strikeout  b_walk  b_k_percent  b_bb_percent  batting_avg  \
0               24       0         37.5           0.0        0.138   
1              105      35         18.5           6.2        0.240   
2               95      77         15.5          12.5        0.273   
3              145      84         23.4          13.5        0.250   
4               68      31         13.2           6.0        0.246   
...            ...     ...          ...           ...          ...   
10904           24       0         41.4           0.0        0.082   
10905           78      63         18.3          14.8        0.233   
10906           78      63         18.3          14.8        0.233   
10907            2       2         12.5          12.5        0.214   
10908            4       1         40.0          10.0        0.111   

       slg_percent  on_base_percent  on_base_plus_slg  
0            0.155            0.141             0.296  
1            0.409            0.293             0.702  
2            0.553            0.360             0.913  
3            0.486            0.356             0.842  
4            0.423            0.297             0.720  
...            ...              ...               ...  
10904        0.082            0.069             0.151  
10905        0.378            0.344             0.722  
10906        0.378            0.344             0.722  
10907        0.286            0.313             0.598  
10908        0.111            0.200             0.311  

[10909 rows x 23 columns]

In [51]:
complete_data_and_war.drop_duplicates()

full_name  player_id  year  player_age league  Total_WAR  \
0          Bartolo Colon   112526.0  2015          42     NL       1.22   
1           Torii Hunter   116338.0  2015          40     AL      -0.65   
2            David Ortiz   120074.0  2015          40     AL       3.10   
3         Alex Rodriguez   121347.0  2015          40     AL       2.95   
4         Aramis Ramirez   133380.0  2015          37     NL       0.00   
...                  ...        ...   ...         ...    ...        ...   
10904  Roberto Hernandez   433584.0  2014          34     NL      -0.79   
10905          Ike Davis   477195.0  2014          27     NL       0.34   
10906          Ike Davis   477195.0  2014          27     NL      -0.07   
10907    Henry Rodriguez   514719.0  2012          22     NL      -0.04   
10908    Henry Rodriguez   514719.0  2013          23     NL      -0.11   

       WAR_bat  WAR_pit  b_ab  b_total_pa  ...  b_triple  b_home_run  \
0        -0.07     1.29    58          64  ...         0           0   
1        -0.65     0.00   521         567  ...         0          22   
2         3.10     0.00   528         614  ...         0          37   
3         2.95     0.00   523         620  ...         1          33   
4         0.00     0.00   475         516  ...         1          17   
...        ...      ...   ...         ...  ...       ...         ...   
10904    -0.38    -0.41    49          58  ...         0           0   
10905     0.34     0.00   360         427  ...         0          11   
10906    -0.07     0.00   360         427  ...         0          11   
10907    -0.04     0.00    14          16  ...         0           0   
10908    -0.11     0.00     9          10  ...         0           0   

       b_strikeout  b_walk  b_k_percent  b_bb_percent  batting_avg  \
0               24       0         37.5           0.0        0.138   
1              105      35         18.5           6.2        0.240   
2               95      77         15.5          12.5        0.273   
3              145      84         23.4          13.5        0.250   
4               68      31         13.2           6.0        0.246   
...            ...     ...          ...           ...          ...   
10904           24       0         41.4           0.0        0.082   
10905           78      63         18.3          14.8        0.233   
10906           78      63         18.3          14.8        0.233   
10907            2       2         12.5          12.5        0.214   
10908            4       1         40.0          10.0        0.111   

       slg_percent  on_base_percent  on_base_plus_slg  
0            0.155            0.141             0.296  
1            0.409            0.293             0.702  
2            0.553            0.360             0.913  
3            0.486            0.356             0.842  
4            0.423            0.297             0.720  
...            ...              ...               ...  
10904        0.082            0.069             0.151  
10905        0.378            0.344             0.722  
10906        0.378            0.344             0.722  
10907        0.286            0.313             0.598  
10908        0.111            0.200             0.311  

[10891 rows x 23 columns]

In [52]:
complete_data_and_war.isnull().sum()

full_name           0
player_id           0
year                0
player_age          0
league              8
Total_WAR           8
WAR_bat             8
WAR_pit             8
b_ab                0
b_total_pa          0
b_total_hits        0
b_single            0
b_double            0
b_triple            0
b_home_run          0
b_strikeout         0
b_walk              0
b_k_percent         0
b_bb_percent        0
batting_avg         0
slg_percent         0
on_base_percent     0
on_base_plus_slg    0
dtype: int64

There are some players that played for different team during a same season. To our model, we will pick the highest total war to chose between the data available.

In [53]:
complete_data_and_war = complete_data_and_war.sort_values(by = 'Total_WAR', ascending = False)
complete_data_and_war

full_name  player_id  year  player_age league  Total_WAR  WAR_bat  \
2904  Mookie Betts   605141.0  2018          26     AL      10.68    10.68   
8027    Mike Trout   545361.0  2012          21     AL      10.54    10.54   
1318    Mike Trout   545361.0  2016          25     AL      10.48    10.48   
2846  Jacob deGrom   594798.0  2018          30     NL      10.36     0.46   
2697    Mike Trout   545361.0  2018          27     AL       9.92     9.92   
...            ...        ...   ...         ...    ...        ...      ...   
3479  A.J. Pollock   572041.0  2019          32    NaN        NaN      NaN   
4039  A.J. Pollock   572041.0  2020          33    NaN        NaN      NaN   
4760  A.J. Pollock   572041.0  2021          34    NaN        NaN      NaN   
8015  A.J. Pollock   572041.0  2012          25    NaN        NaN      NaN   
9395  A.J. Pollock   572041.0  2014          27    NaN        NaN      NaN   

      WAR_pit  b_ab  b_total_pa  ...  b_triple  b_home_run  b_strikeout  \
2904      0.0   520         614  ...         5          32           91   
8027      0.0   559         639  ...         8          30          139   
1318      0.0   549         681  ...         5          29          137   
2846      9.9    67          74  ...         0           0           25   
2697      0.0   471         608  ...         4          39          124   
...       ...   ...         ...  ...       ...         ...          ...   
3479      NaN   308         342  ...         1          15           74   
4039      NaN   196         210  ...         0          16           45   
4760      NaN   384         422  ...         1          21           80   
8015      NaN    81          93  ...         1           2           11   
9395      NaN   265         287  ...         6           7           46   

      b_walk  b_k_percent  b_bb_percent  batting_avg  slg_percent  \
2904      81         14.8          13.2        0.346        0.640   
8027      67         21.8          10.5        0.326        0.564   
1318     116         20.1          17.0        0.315        0.550   
2846       4         33.8           5.4        0.164        0.179   
2697     122         20.4          20.1        0.312        0.628   
...      ...          ...           ...          ...          ...   
3479      23         21.6           6.7        0.266        0.468   
4039      12         21.4           5.7        0.276        0.566   
4760      30         19.0           7.1        0.297        0.536   
8015       9         11.8           9.7        0.247        0.395   
9395      19         16.0           6.6        0.302        0.498   

      on_base_percent  on_base_plus_slg  
2904            0.438             1.078  
8027            0.399             0.963  
1318            0.441             0.991  
2846            0.203             0.382  
2697            0.459             1.087  
...               ...               ...  
3479            0.327             0.795  
4039            0.314             0.881  
4760            0.355             0.892  
8015            0.312             0.707  
9395            0.352             0.850  

[10909 rows x 23 columns]

In [54]:
complete_data_and_war = complete_data_and_war.drop_duplicates(subset=['full_name','player_id','year'], keep='first')
complete_data_and_war

full_name  player_id  year  player_age league  Total_WAR  WAR_bat  \
2904  Mookie Betts   605141.0  2018          26     AL      10.68    10.68   
8027    Mike Trout   545361.0  2012          21     AL      10.54    10.54   
1318    Mike Trout   545361.0  2016          25     AL      10.48    10.48   
2846  Jacob deGrom   594798.0  2018          30     NL      10.36     0.46   
2697    Mike Trout   545361.0  2018          27     AL       9.92     9.92   
...            ...        ...   ...         ...    ...        ...      ...   
3479  A.J. Pollock   572041.0  2019          32    NaN        NaN      NaN   
4039  A.J. Pollock   572041.0  2020          33    NaN        NaN      NaN   
4760  A.J. Pollock   572041.0  2021          34    NaN        NaN      NaN   
8015  A.J. Pollock   572041.0  2012          25    NaN        NaN      NaN   
9395  A.J. Pollock   572041.0  2014          27    NaN        NaN      NaN   

      WAR_pit  b_ab  b_total_pa  ...  b_triple  b_home_run  b_strikeout  \
2904      0.0   520         614  ...         5          32           91   
8027      0.0   559         639  ...         8          30          139   
1318      0.0   549         681  ...         5          29          137   
2846      9.9    67          74  ...         0           0           25   
2697      0.0   471         608  ...         4          39          124   
...       ...   ...         ...  ...       ...         ...          ...   
3479      NaN   308         342  ...         1          15           74   
4039      NaN   196         210  ...         0          16           45   
4760      NaN   384         422  ...         1          21           80   
8015      NaN    81          93  ...         1           2           11   
9395      NaN   265         287  ...         6           7           46   

      b_walk  b_k_percent  b_bb_percent  batting_avg  slg_percent  \
2904      81         14.8          13.2        0.346        0.640   
8027      67         21.8          10.5        0.326        0.564   
1318     116         20.1          17.0        0.315        0.550   
2846       4         33.8           5.4        0.164        0.179   
2697     122         20.4          20.1        0.312        0.628   
...      ...          ...           ...          ...          ...   
3479      23         21.6           6.7        0.266        0.468   
4039      12         21.4           5.7        0.276        0.566   
4760      30         19.0           7.1        0.297        0.536   
8015       9         11.8           9.7        0.247        0.395   
9395      19         16.0           6.6        0.302        0.498   

      on_base_percent  on_base_plus_slg  
2904            0.438             1.078  
8027            0.399             0.963  
1318            0.441             0.991  
2846            0.203             0.382  
2697            0.459             1.087  
...               ...               ...  
3479            0.327             0.795  
4039            0.314             0.881  
4760            0.355             0.892  
8015            0.312             0.707  
9395            0.352             0.850  

[9781 rows x 23 columns]

In [55]:
complete_data_and_war.isnull().sum()

full_name           0
player_id           0
year                0
player_age          0
league              8
Total_WAR           8
WAR_bat             8
WAR_pit             8
b_ab                0
b_total_pa          0
b_total_hits        0
b_single            0
b_double            0
b_triple            0
b_home_run          0
b_strikeout         0
b_walk              0
b_k_percent         0
b_bb_percent        0
batting_avg         0
slg_percent         0
on_base_percent     0
on_base_plus_slg    0
dtype: int64

### Adding the MVP Votes and Ranking

In [56]:
for year in range(2008,2022):

    NL_df = pd.read_csv(f'MVP Votes Raw Data/NL_{year}.csv')
    AL_df = pd.read_csv(f'MVP Votes Raw Data/AL_{year}.csv')

    ref = [NL_df, AL_df]
    df_comb = pd.concat(ref)
    cols = ['first_name', 'last_name']
    df_comb['full_name'] = df_comb[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
    df_comb['full_name'] = df_comb['full_name'].apply(unidecode)
    df_comb['year'] = year
    df_comb.sort_values(by = 'rank')
    
   # df_comb.to_csv(f'MVP Cleaned Data/{year}_mvp.csv')


In [57]:
path = 'MVP Cleaned Data'
all_files = glob.glob(path + "/*_mvp.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

voted_for_mvps = pd.concat(li, axis=0, ignore_index=True)

In [58]:
voted_for_mvps = voted_for_mvps[['full_name','year','rank','vote_points','1st_place','mvp_share']]

We are considering a timeframe of 14 years. Therefore, we should expect 28 players ranked as 1st. 

In [59]:
voted_for_mvps.groupby('rank')['rank'].count()

rank
1     28
2     28
3     28
4     28
5     28
6     29
7     28
8     27
9     29
10    28
11    27
12    30
13    29
14    30
15    33
16    28
17    33
18    27
19    30
20    35
21    23
22    19
23    14
24    12
25     9
26     7
27     1
28    11
30     2
32     4
Name: rank, dtype: int64

In [60]:
stats_war_mvp_data = pd.merge(complete_data_and_war, voted_for_mvps, how = 'left', on=['full_name','year'])
stats_war_mvp_data = stats_war_mvp_data[['full_name', 'player_id', 'year', 'player_age', 'league', 'rank', 'vote_points',
       '1st_place', 'mvp_share','Total_WAR','WAR_bat', 'WAR_pit', 'b_ab', 'b_total_pa', 'b_total_hits', 'b_single',
       'b_double', 'b_triple', 'b_home_run', 'b_strikeout', 'b_walk',
       'b_k_percent', 'b_bb_percent', 'batting_avg', 'slg_percent',
       'on_base_percent', 'on_base_plus_slg']]
stats_war_mvp_data

full_name  player_id  year  player_age league  rank  vote_points  \
0     Mookie Betts   605141.0  2018          26     AL   1.0        410.0   
1       Mike Trout   545361.0  2012          21     AL   2.0        281.0   
2       Mike Trout   545361.0  2016          25     AL   1.0        356.0   
3     Jacob deGrom   594798.0  2018          30     NL   5.0        141.0   
4       Mike Trout   545361.0  2018          27     AL   2.0        265.0   
...            ...        ...   ...         ...    ...   ...          ...   
9776  A.J. Pollock   572041.0  2019          32    NaN   NaN          NaN   
9777  A.J. Pollock   572041.0  2020          33    NaN   NaN          NaN   
9778  A.J. Pollock   572041.0  2021          34    NaN   NaN          NaN   
9779  A.J. Pollock   572041.0  2012          25    NaN   NaN          NaN   
9780  A.J. Pollock   572041.0  2014          27    NaN   NaN          NaN   

      1st_place  mvp_share  Total_WAR  ...  b_triple  b_home_run  b_strikeout  \
0          28.0       0.98      10.68  ...         5          32           91   
1           6.0       0.72      10.54  ...         8          30          139   
2          19.0       0.85      10.48  ...         5          29          137   
3           1.0       0.34      10.36  ...         0           0           25   
4           1.0       0.63       9.92  ...         4          39          124   
...         ...        ...        ...  ...       ...         ...          ...   
9776        NaN        NaN        NaN  ...         1          15           74   
9777        NaN        NaN        NaN  ...         0          16           45   
9778        NaN        NaN        NaN  ...         1          21           80   
9779        NaN        NaN        NaN  ...         1           2           11   
9780        NaN        NaN        NaN  ...         6           7           46   

      b_walk  b_k_percent  b_bb_percent  batting_avg  slg_percent  \
0         81         14.8          13.2        0.346        0.640   
1         67         21.8          10.5        0.326        0.564   
2        116         20.1          17.0        0.315        0.550   
3          4         33.8           5.4        0.164        0.179   
4        122         20.4          20.1        0.312        0.628   
...      ...          ...           ...          ...          ...   
9776      23         21.6           6.7        0.266        0.468   
9777      12         21.4           5.7        0.276        0.566   
9778      30         19.0           7.1        0.297        0.536   
9779       9         11.8           9.7        0.247        0.395   
9780      19         16.0           6.6        0.302        0.498   

      on_base_percent  on_base_plus_slg  
0               0.438             1.078  
1               0.399             0.963  
2               0.441             0.991  
3               0.203             0.382  
4               0.459             1.087  
...               ...               ...  
9776            0.327             0.795  
9777            0.314             0.881  
9778            0.355             0.892  
9779            0.312             0.707  
9780            0.352             0.850  

[9781 rows x 27 columns]

However, our stats dataset only accounts for batters. During this timeframe, only one pitcher has been awarded as MVP, in 2011. Therefore, we should find 27 #1 in our set. Let's check:

In [61]:
stats_war_mvp_data.groupby('rank')['rank'].count()

rank
1.0     27
2.0     28
3.0     26
4.0     27
5.0     27
6.0     27
7.0     27
8.0     24
9.0     26
10.0    23
11.0    23
12.0    25
13.0    24
14.0    24
15.0    27
16.0    24
17.0    27
18.0    21
19.0    22
20.0    32
21.0    19
22.0    12
23.0    13
24.0    11
25.0     7
26.0     6
27.0     1
28.0     9
30.0     2
32.0     4
Name: rank, dtype: int64

Now, let's assign 0 for all players that haven't been voted.

In [62]:
stats_war_mvp_data['rank'] = stats_war_mvp_data['rank'].fillna(0)
stats_war_mvp_data['vote_points'] = stats_war_mvp_data['vote_points'].fillna(0)
stats_war_mvp_data['1st_place'] = stats_war_mvp_data['1st_place'].fillna(0)
stats_war_mvp_data['mvp_share'] = stats_war_mvp_data['mvp_share'].fillna(0)

In [63]:
stats_war_mvp_data

full_name  player_id  year  player_age league  rank  vote_points  \
0     Mookie Betts   605141.0  2018          26     AL   1.0        410.0   
1       Mike Trout   545361.0  2012          21     AL   2.0        281.0   
2       Mike Trout   545361.0  2016          25     AL   1.0        356.0   
3     Jacob deGrom   594798.0  2018          30     NL   5.0        141.0   
4       Mike Trout   545361.0  2018          27     AL   2.0        265.0   
...            ...        ...   ...         ...    ...   ...          ...   
9776  A.J. Pollock   572041.0  2019          32    NaN   0.0          0.0   
9777  A.J. Pollock   572041.0  2020          33    NaN   0.0          0.0   
9778  A.J. Pollock   572041.0  2021          34    NaN   0.0          0.0   
9779  A.J. Pollock   572041.0  2012          25    NaN   0.0          0.0   
9780  A.J. Pollock   572041.0  2014          27    NaN   0.0          0.0   

      1st_place  mvp_share  Total_WAR  ...  b_triple  b_home_run  b_strikeout  \
0          28.0       0.98      10.68  ...         5          32           91   
1           6.0       0.72      10.54  ...         8          30          139   
2          19.0       0.85      10.48  ...         5          29          137   
3           1.0       0.34      10.36  ...         0           0           25   
4           1.0       0.63       9.92  ...         4          39          124   
...         ...        ...        ...  ...       ...         ...          ...   
9776        0.0       0.00        NaN  ...         1          15           74   
9777        0.0       0.00        NaN  ...         0          16           45   
9778        0.0       0.00        NaN  ...         1          21           80   
9779        0.0       0.00        NaN  ...         1           2           11   
9780        0.0       0.00        NaN  ...         6           7           46   

      b_walk  b_k_percent  b_bb_percent  batting_avg  slg_percent  \
0         81         14.8          13.2        0.346        0.640   
1         67         21.8          10.5        0.326        0.564   
2        116         20.1          17.0        0.315        0.550   
3          4         33.8           5.4        0.164        0.179   
4        122         20.4          20.1        0.312        0.628   
...      ...          ...           ...          ...          ...   
9776      23         21.6           6.7        0.266        0.468   
9777      12         21.4           5.7        0.276        0.566   
9778      30         19.0           7.1        0.297        0.536   
9779       9         11.8           9.7        0.247        0.395   
9780      19         16.0           6.6        0.302        0.498   

      on_base_percent  on_base_plus_slg  
0               0.438             1.078  
1               0.399             0.963  
2               0.441             0.991  
3               0.203             0.382  
4               0.459             1.087  
...               ...               ...  
9776            0.327             0.795  
9777            0.314             0.881  
9778            0.355             0.892  
9779            0.312             0.707  
9780            0.352             0.850  

[9781 rows x 27 columns]

## Final data analysis

Now, before closing the data cleaning, we need to remove some useless data. We can observe that some rows don't have WAR information, which is a crucial piece of data for our analysis.
Therefore, we shall drop the rows without this information, as we already added our WAR values.

In [64]:
stats_war_mvp_data = stats_war_mvp_data[stats_war_mvp_data['Total_WAR'].notna()]
stats_war_mvp_data.isnull().sum()

full_name           0
player_id           0
year                0
player_age          0
league              0
rank                0
vote_points         0
1st_place           0
mvp_share           0
Total_WAR           0
WAR_bat             0
WAR_pit             0
b_ab                0
b_total_pa          0
b_total_hits        0
b_single            0
b_double            0
b_triple            0
b_home_run          0
b_strikeout         0
b_walk              0
b_k_percent         0
b_bb_percent        0
batting_avg         0
slg_percent         0
on_base_percent     0
on_base_plus_slg    0
dtype: int64

Great! Nothing missing!

### Spliting the final dataset into separated datasets

In [65]:
for year in range(2008,2022):
    
    export_data = stats_war_mvp_data.loc[stats_war_mvp_data['year'] == year]
   # export_data.to_csv(f'./Complete/{year}_complete.csv')

## Adding the 2022 players available information

In [66]:
new_year = 2022

df_AL = pd.read_csv(f'Statcast Raw Data/{new_year}_AL.csv')
df_NL = pd.read_csv(f'Statcast Raw Data/{new_year}_NL.csv')

From these Datasets, we don't have much relevant data. Let's filter our data:

In [67]:
df_AL = df_AL[['player_name','player_id','abs','hits']]
df_NL = df_NL[['player_name','player_id','abs','hits']]

### 2022 AL League players age finder

In [68]:
df_AL

player_name  player_id  abs  hits
0         Murphy, Tom     608596    3     2
1        Kwan, Steven     680757   15    10
2        Choi, Ji-Man     596847   13     8
3         Pollock, AJ     572041    7     4
4       Jansen, Danny     643376    7     4
..                ...        ...  ...   ...
192     Sheets, Gavin     657757    6     0
193  Gonzalez, Marwin     503556    1     0
194      Smith, Kevin     675656   12     0
195         Chang, Yu     644374    7     0
196      Moore, Dylan     664238    0     0

[197 rows x 4 columns]

#### Using the bat war dataset as out players index

In [69]:
player_id_name_age = bat_data_war[['full_name','player_id','player_age','year']]
player_id_name_age = player_id_name_age.rename({'player_age':'player_age_war_db','year':'year_war_db'}, axis=1)
player_id_name_age = player_id_name_age.sort_values(by = 'player_age_war_db', ascending = False)
player_id_name_age2 = player_id_name_age.drop_duplicates(subset = 'player_id', keep = 'first')
player_id_name_age2

full_name  player_id  player_age_war_db  year_war_db
75367       Jamie Moyer   119469.0               49.0         2012
53818     Randy Johnson   116615.0               45.0         2009
104201    Ichiro Suzuki   400085.0               45.0         2019
110393     Omar Vizquel   123744.0               45.0         2012
20868     Bartolo Colon   112526.0               45.0         2018
...                 ...        ...                ...          ...
66211   Brailyn Marquez   666622.0               21.0         2020
81896       Luis Patino   672715.0               20.0         2020
75729      Andres Munoz   662253.0               20.0         2019
35162     Wander Franco   677551.0               20.0         2021
63971     Elvis Luciano   672773.0               19.0         2019

[4426 rows x 4 columns]

In [70]:
player_age_lookup_merge_AL = pd.merge(df_AL, player_id_name_age2, how = 'left', on=['player_id'])
player_age_lookup_merge_AL

player_name  player_id  abs  hits        full_name  \
0         Murphy, Tom     608596    3     2       Tom Murphy   
1        Kwan, Steven     680757   15    10              NaN   
2        Choi, Ji-Man     596847   13     8      Ji-Man Choi   
3         Pollock, AJ     572041    7     4       AJ Pollock   
4       Jansen, Danny     643376    7     4     Danny Jansen   
..                ...        ...  ...   ...              ...   
192     Sheets, Gavin     657757    6     0     Gavin Sheets   
193  Gonzalez, Marwin     503556    1     0  Marwin Gonzalez   
194      Smith, Kevin     675656   12     0      Kevin Smith   
195         Chang, Yu     644374    7     0         Yu Chang   
196      Moore, Dylan     664238    0     0      Dylan Moore   

     player_age_war_db  year_war_db  
0                 30.0       2021.0  
1                  NaN          NaN  
2                 30.0       2021.0  
3                 33.0       2021.0  
4                 26.0       2021.0  
..                 ...          ...  
192               25.0       2021.0  
193               32.0       2021.0  
194               24.0       2021.0  
195               25.0       2021.0  
196               28.0       2021.0  

[197 rows x 7 columns]

In [71]:
player_age_lookup_merge_AL['year'] = new_year
player_age_lookup_merge_AL['years_diff'] = player_age_lookup_merge_AL['year']-player_age_lookup_merge_AL['year_war_db']
player_age_lookup_merge_AL['player_age'] = player_age_lookup_merge_AL['player_age_war_db'] + player_age_lookup_merge_AL['years_diff']
player_age_lookup_merge_AL

player_name  player_id  abs  hits        full_name  \
0         Murphy, Tom     608596    3     2       Tom Murphy   
1        Kwan, Steven     680757   15    10              NaN   
2        Choi, Ji-Man     596847   13     8      Ji-Man Choi   
3         Pollock, AJ     572041    7     4       AJ Pollock   
4       Jansen, Danny     643376    7     4     Danny Jansen   
..                ...        ...  ...   ...              ...   
192     Sheets, Gavin     657757    6     0     Gavin Sheets   
193  Gonzalez, Marwin     503556    1     0  Marwin Gonzalez   
194      Smith, Kevin     675656   12     0      Kevin Smith   
195         Chang, Yu     644374    7     0         Yu Chang   
196      Moore, Dylan     664238    0     0      Dylan Moore   

     player_age_war_db  year_war_db  year  years_diff  player_age  
0                 30.0       2021.0  2022         1.0        31.0  
1                  NaN          NaN  2022         NaN         NaN  
2                 30.0       2021.0  2022         1.0        31.0  
3                 33.0       2021.0  2022         1.0        34.0  
4                 26.0       2021.0  2022         1.0        27.0  
..                 ...          ...   ...         ...         ...  
192               25.0       2021.0  2022         1.0        26.0  
193               32.0       2021.0  2022         1.0        33.0  
194               24.0       2021.0  2022         1.0        25.0  
195               25.0       2021.0  2022         1.0        26.0  
196               28.0       2021.0  2022         1.0        29.0  

[197 rows x 10 columns]

In [72]:
player_age_lookup_merge_AL_nan = player_age_lookup_merge_AL[player_age_lookup_merge_AL['player_age_war_db'].isna()]
player_age_lookup_merge_AL_nan


list_of_pl_AL = player_age_lookup_merge_AL_nan[['player_id','player_name']]
list_of_pl_AL

player_id         player_name
1       680757        Kwan, Steven
56      665161        Peña, Jeremy
143     677951     Witt Jr., Bobby
154     679529  Torkelson, Spencer
157     677594    Rodríguez, Julio
170     669309          Perez, Joe
187     681526    Lavastida, Bryan

In [73]:
player_age_lookup_merge_AL.loc[player_age_lookup_merge_AL['player_id'] == 680757, 'full_name'] = 'Steven Kwan'
player_age_lookup_merge_AL.loc[player_age_lookup_merge_AL['player_id'] == 680757, 'player_age'] = 25

player_age_lookup_merge_AL.loc[player_age_lookup_merge_AL['player_id'] == 665161, 'full_name'] = 'Jeremy Pena'
player_age_lookup_merge_AL.loc[player_age_lookup_merge_AL['player_id'] == 665161, 'player_age'] = 25

player_age_lookup_merge_AL.loc[player_age_lookup_merge_AL['player_id'] == 677951, 'full_name'] = 'Bobby Witt Jr.'
player_age_lookup_merge_AL.loc[player_age_lookup_merge_AL['player_id'] == 677951, 'player_age'] = 22

player_age_lookup_merge_AL.loc[player_age_lookup_merge_AL['player_id'] == 679529, 'full_name'] = 'Spencer Torkelson'
player_age_lookup_merge_AL.loc[player_age_lookup_merge_AL['player_id'] == 679529, 'player_age'] = 23

player_age_lookup_merge_AL.loc[player_age_lookup_merge_AL['player_id'] == 677594, 'full_name'] = 'Julio Rodriguez'
player_age_lookup_merge_AL.loc[player_age_lookup_merge_AL['player_id'] == 677594, 'player_age'] = 22

player_age_lookup_merge_AL.loc[player_age_lookup_merge_AL['player_id'] == 669309, 'full_name'] = 'Joe Perez'
player_age_lookup_merge_AL.loc[player_age_lookup_merge_AL['player_id'] == 669309, 'player_age'] = 23

player_age_lookup_merge_AL.loc[player_age_lookup_merge_AL['player_id'] == 681526, 'full_name'] = 'Bryan Lavastida'
player_age_lookup_merge_AL.loc[player_age_lookup_merge_AL['player_id'] == 681526, 'player_age'] = 24

player_age_lookup_merge_AL_final = player_age_lookup_merge_AL[['full_name','player_id','year','player_age']]
player_age_lookup_merge_AL_final['league'] = 'AL'
player_age_lookup_merge_AL_final

C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_12108/3214079982.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_age_lookup_merge_AL_final['league'] = 'AL'


full_name  player_id  year  player_age league
0         Tom Murphy     608596  2022        31.0     AL
1        Steven Kwan     680757  2022        25.0     AL
2        Ji-Man Choi     596847  2022        31.0     AL
3         AJ Pollock     572041  2022        34.0     AL
4       Danny Jansen     643376  2022        27.0     AL
..               ...        ...   ...         ...    ...
192     Gavin Sheets     657757  2022        26.0     AL
193  Marwin Gonzalez     503556  2022        33.0     AL
194      Kevin Smith     675656  2022        25.0     AL
195         Yu Chang     644374  2022        26.0     AL
196      Dylan Moore     664238  2022        29.0     AL

[197 rows x 5 columns]

In [74]:
list_of_cols = ['full_name', 'player_id', 'year', 'player_age','league',
       'Total_WAR','WAR_bat', 'WAR_pit', 'b_ab', 'b_total_pa',
       'b_total_hits', 'b_single', 'b_double', 'b_triple', 'b_home_run',
       'b_strikeout', 'b_walk', 'b_k_percent', 'b_bb_percent', 'batting_avg',
       'slg_percent', 'on_base_percent', 'on_base_plus_slg']

player_age_lookup_merge_AL_final = player_age_lookup_merge_AL_final.reindex(columns = list_of_cols)
player_age_lookup_merge_AL_final

full_name  player_id  year  player_age league  Total_WAR  WAR_bat  \
0         Tom Murphy     608596  2022        31.0     AL        NaN      NaN   
1        Steven Kwan     680757  2022        25.0     AL        NaN      NaN   
2        Ji-Man Choi     596847  2022        31.0     AL        NaN      NaN   
3         AJ Pollock     572041  2022        34.0     AL        NaN      NaN   
4       Danny Jansen     643376  2022        27.0     AL        NaN      NaN   
..               ...        ...   ...         ...    ...        ...      ...   
192     Gavin Sheets     657757  2022        26.0     AL        NaN      NaN   
193  Marwin Gonzalez     503556  2022        33.0     AL        NaN      NaN   
194      Kevin Smith     675656  2022        25.0     AL        NaN      NaN   
195         Yu Chang     644374  2022        26.0     AL        NaN      NaN   
196      Dylan Moore     664238  2022        29.0     AL        NaN      NaN   

     WAR_pit  b_ab  b_total_pa  ...  b_triple  b_home_run  b_strikeout  \
0        NaN   NaN         NaN  ...       NaN         NaN          NaN   
1        NaN   NaN         NaN  ...       NaN         NaN          NaN   
2        NaN   NaN         NaN  ...       NaN         NaN          NaN   
3        NaN   NaN         NaN  ...       NaN         NaN          NaN   
4        NaN   NaN         NaN  ...       NaN         NaN          NaN   
..       ...   ...         ...  ...       ...         ...          ...   
192      NaN   NaN         NaN  ...       NaN         NaN          NaN   
193      NaN   NaN         NaN  ...       NaN         NaN          NaN   
194      NaN   NaN         NaN  ...       NaN         NaN          NaN   
195      NaN   NaN         NaN  ...       NaN         NaN          NaN   
196      NaN   NaN         NaN  ...       NaN         NaN          NaN   

     b_walk  b_k_percent  b_bb_percent  batting_avg  slg_percent  \
0       NaN          NaN           NaN          NaN          NaN   
1       NaN          NaN           NaN          NaN          NaN   
2       NaN          NaN           NaN          NaN          NaN   
3       NaN          NaN           NaN          NaN          NaN   
4       NaN          NaN           NaN          NaN          NaN   
..      ...          ...           ...          ...          ...   
192     NaN          NaN           NaN          NaN          NaN   
193     NaN          NaN           NaN          NaN          NaN   
194     NaN          NaN           NaN          NaN          NaN   
195     NaN          NaN           NaN          NaN          NaN   
196     NaN          NaN           NaN          NaN          NaN   

     on_base_percent  on_base_plus_slg  
0                NaN               NaN  
1                NaN               NaN  
2                NaN               NaN  
3                NaN               NaN  
4                NaN               NaN  
..               ...               ...  
192              NaN               NaN  
193              NaN               NaN  
194              NaN               NaN  
195              NaN               NaN  
196              NaN               NaN  

[197 rows x 23 columns]

### 2022 NL League players age finder

In [75]:
df_NL

player_name  player_id  abs  hits
0    De La Cruz, Bryan     650559    1     1
1           Bohm, Alec     664761    4     3
2            Happ, Ian     664023   10     7
3       Barnes, Austin     605131    3     2
4       Daza, Yonathan     602074    4     2
..                 ...        ...  ...   ...
191    Guillorme, Luis     641645    8     0
192     Garcia, Aramis     605244    2     0
193      Herrera, Jose     645444    5     0
194      Henry, Payton     668721    3     0
195   Hernandez, Yonny     660634    0     0

[196 rows x 4 columns]

In [76]:
player_age_lookup_merge_NL = pd.merge(df_NL, player_id_name_age2, how = 'left', on=['player_id'])
player_age_lookup_merge_NL

player_name  player_id  abs  hits         full_name  \
0    De La Cruz, Bryan     650559    1     1  Bryan De La Cruz   
1           Bohm, Alec     664761    4     3         Alec Bohm   
2            Happ, Ian     664023   10     7          Ian Happ   
3       Barnes, Austin     605131    3     2     Austin Barnes   
4       Daza, Yonathan     602074    4     2     Yonathan Daza   
..                 ...        ...  ...   ...               ...   
191    Guillorme, Luis     641645    8     0    Luis Guillorme   
192     Garcia, Aramis     605244    2     0     Aramis Garcia   
193      Herrera, Jose     645444    5     0               NaN   
194      Henry, Payton     668721    3     0      Payton Henry   
195   Hernandez, Yonny     660634    0     0   Yonny Hernandez   

     player_age_war_db  year_war_db  
0                 24.0       2021.0  
1                 24.0       2021.0  
2                 26.0       2021.0  
3                 31.0       2021.0  
4                 27.0       2021.0  
..                 ...          ...  
191               26.0       2021.0  
192               28.0       2021.0  
193                NaN          NaN  
194               24.0       2021.0  
195               23.0       2021.0  

[196 rows x 7 columns]

In [77]:
player_age_lookup_merge_NL['year'] = new_year
player_age_lookup_merge_NL['years_diff'] = player_age_lookup_merge_NL['year']-player_age_lookup_merge_NL['year_war_db']
player_age_lookup_merge_NL['player_age'] = player_age_lookup_merge_NL['player_age_war_db'] + player_age_lookup_merge_NL['years_diff']
player_age_lookup_merge_NL

player_name  player_id  abs  hits         full_name  \
0    De La Cruz, Bryan     650559    1     1  Bryan De La Cruz   
1           Bohm, Alec     664761    4     3         Alec Bohm   
2            Happ, Ian     664023   10     7          Ian Happ   
3       Barnes, Austin     605131    3     2     Austin Barnes   
4       Daza, Yonathan     602074    4     2     Yonathan Daza   
..                 ...        ...  ...   ...               ...   
191    Guillorme, Luis     641645    8     0    Luis Guillorme   
192     Garcia, Aramis     605244    2     0     Aramis Garcia   
193      Herrera, Jose     645444    5     0               NaN   
194      Henry, Payton     668721    3     0      Payton Henry   
195   Hernandez, Yonny     660634    0     0   Yonny Hernandez   

     player_age_war_db  year_war_db  year  years_diff  player_age  
0                 24.0       2021.0  2022         1.0        25.0  
1                 24.0       2021.0  2022         1.0        25.0  
2                 26.0       2021.0  2022         1.0        27.0  
3                 31.0       2021.0  2022         1.0        32.0  
4                 27.0       2021.0  2022         1.0        28.0  
..                 ...          ...   ...         ...         ...  
191               26.0       2021.0  2022         1.0        27.0  
192               28.0       2021.0  2022         1.0        29.0  
193                NaN          NaN  2022         NaN         NaN  
194               24.0       2021.0  2022         1.0        25.0  
195               23.0       2021.0  2022         1.0        24.0  

[196 rows x 10 columns]

In [78]:
player_age_lookup_merge_NL_nan = player_age_lookup_merge_NL[player_age_lookup_merge_NL['player_age_war_db'].isna()]
player_age_lookup_merge_NL_nan


list_of_pl_NL = player_age_lookup_merge_NL_nan[['player_id','player_name']]
list_of_pl_NL

player_id       player_name
7       671218     Ramos, Heliot
12      640492      Azocar, José
21      673548     Suzuki, Seiya
24      660636   Castillo, Diego
113     681082     Stott, Bryson
158     669450    Hummel, Cooper
159     682928      Abrams, C.J.
175     665922  Muzziotti, Símon
189     665650       Fox, Lucius
193     645444     Herrera, Jose

In [79]:
player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 671218, 'full_name'] = 'Heliot Ramos'
player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 671218, 'player_age'] = 23

player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 640492, 'full_name'] = 'Jose Azocar'
player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 640492, 'player_age'] = 26

player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 673548, 'full_name'] = 'Seiya Suzuki'
player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 673548, 'player_age'] = 28

player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 681082, 'full_name'] = 'Diego Castillo'
player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 681082, 'player_age'] = 25

player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 669450, 'full_name'] = 'Bryson Stott'
player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 669450, 'player_age'] = 25

player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 682928, 'full_name'] = 'Cooper Hummel'
player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 682928, 'player_age'] = 28

player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 665922, 'full_name'] = 'C.J. Abrams'
player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 665922, 'player_age'] = 22

player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 665650, 'full_name'] = 'Simon Muzziotti'
player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 665650, 'player_age'] = 24

player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 645444, 'full_name'] = 'Lucius Fox'
player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 645444, 'player_age'] = 25

player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 660636, 'full_name'] = 'Jose Herrera'
player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 660636, 'player_age'] = 25

player_age_lookup_merge_NL_final = player_age_lookup_merge_NL[['full_name','player_id','year','player_age']]
player_age_lookup_merge_NL_final['league'] = 'NL'
player_age_lookup_merge_NL_final

C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_12108/3412345940.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_age_lookup_merge_NL_final['league'] = 'NL'


full_name  player_id  year  player_age league
0    Bryan De La Cruz     650559  2022        25.0     NL
1           Alec Bohm     664761  2022        25.0     NL
2            Ian Happ     664023  2022        27.0     NL
3       Austin Barnes     605131  2022        32.0     NL
4       Yonathan Daza     602074  2022        28.0     NL
..                ...        ...   ...         ...    ...
191    Luis Guillorme     641645  2022        27.0     NL
192     Aramis Garcia     605244  2022        29.0     NL
193        Lucius Fox     645444  2022        25.0     NL
194      Payton Henry     668721  2022        25.0     NL
195   Yonny Hernandez     660634  2022        24.0     NL

[196 rows x 5 columns]

In [80]:
player_age_lookup_merge_NL_final = player_age_lookup_merge_NL_final.reindex(columns = list_of_cols)
player_age_lookup_merge_NL_final

full_name  player_id  year  player_age league  Total_WAR  WAR_bat  \
0    Bryan De La Cruz     650559  2022        25.0     NL        NaN      NaN   
1           Alec Bohm     664761  2022        25.0     NL        NaN      NaN   
2            Ian Happ     664023  2022        27.0     NL        NaN      NaN   
3       Austin Barnes     605131  2022        32.0     NL        NaN      NaN   
4       Yonathan Daza     602074  2022        28.0     NL        NaN      NaN   
..                ...        ...   ...         ...    ...        ...      ...   
191    Luis Guillorme     641645  2022        27.0     NL        NaN      NaN   
192     Aramis Garcia     605244  2022        29.0     NL        NaN      NaN   
193        Lucius Fox     645444  2022        25.0     NL        NaN      NaN   
194      Payton Henry     668721  2022        25.0     NL        NaN      NaN   
195   Yonny Hernandez     660634  2022        24.0     NL        NaN      NaN   

     WAR_pit  b_ab  b_total_pa  ...  b_triple  b_home_run  b_strikeout  \
0        NaN   NaN         NaN  ...       NaN         NaN          NaN   
1        NaN   NaN         NaN  ...       NaN         NaN          NaN   
2        NaN   NaN         NaN  ...       NaN         NaN          NaN   
3        NaN   NaN         NaN  ...       NaN         NaN          NaN   
4        NaN   NaN         NaN  ...       NaN         NaN          NaN   
..       ...   ...         ...  ...       ...         ...          ...   
191      NaN   NaN         NaN  ...       NaN         NaN          NaN   
192      NaN   NaN         NaN  ...       NaN         NaN          NaN   
193      NaN   NaN         NaN  ...       NaN         NaN          NaN   
194      NaN   NaN         NaN  ...       NaN         NaN          NaN   
195      NaN   NaN         NaN  ...       NaN         NaN          NaN   

     b_walk  b_k_percent  b_bb_percent  batting_avg  slg_percent  \
0       NaN          NaN           NaN          NaN          NaN   
1       NaN          NaN           NaN          NaN          NaN   
2       NaN          NaN           NaN          NaN          NaN   
3       NaN          NaN           NaN          NaN          NaN   
4       NaN          NaN           NaN          NaN          NaN   
..      ...          ...           ...          ...          ...   
191     NaN          NaN           NaN          NaN          NaN   
192     NaN          NaN           NaN          NaN          NaN   
193     NaN          NaN           NaN          NaN          NaN   
194     NaN          NaN           NaN          NaN          NaN   
195     NaN          NaN           NaN          NaN          NaN   

     on_base_percent  on_base_plus_slg  
0                NaN               NaN  
1                NaN               NaN  
2                NaN               NaN  
3                NaN               NaN  
4                NaN               NaN  
..               ...               ...  
191              NaN               NaN  
192              NaN               NaN  
193              NaN               NaN  
194              NaN               NaN  
195              NaN               NaN  

[196 rows x 23 columns]

### Create Final 2022 database

In [81]:
frame_2022 = pd.concat([player_age_lookup_merge_AL_final,player_age_lookup_merge_NL_final])
frame_2022['player_age'] = frame_2022['player_age'].astype(int)

#### Export CSV

In [82]:
#frame_2022.to_csv(f'Complete/{new_year}_complete.csv')